In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

In [3]:
cd ..

C:\Projects\python\recommender


In [10]:
from datasets.torch_movielen import TorchMovielen10k
import torch as T
import torch.nn as nn
import torch.optim as optim

In [11]:
databunch = TorchMovielen10k('./inputs/ml-100k/u.data', user_min=4, item_min=4)
databunch.batch(16)
databunch.device(T.device('cpu'))
databunch.shuffle(False)
databunch.workers(0)

2019-07-16 11:15:02,838 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
2019-07-16 11:15:02,838 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
2019-07-16 11:15:02,838 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
2019-07-16 11:15:02,838 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
2019-07-16 11:15:02,838 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
I0716 11:15:02.838742 12240 torch_movielen.py:45] Read dataset in ./inputs/ml-100k/u.data
2019-07-16 11:15:02,856 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
2019-07-16 11:15:02,856 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
2019-07-16 11:15:02,856 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
2019-07-16 11:15:02,856 - C:\Projects\

In [12]:
train_dl = databunch.get_dataloader(dataset_type='train')
valid_dl = databunch.get_dataloader(dataset_type='valid')
test_dl = databunch.get_dataloader(dataset_type='test')
train_it = iter(train_dl)

In [14]:
pos_batch, neg_batch = train_it.next()

In [15]:
import torch as T
import torch.nn as nn
import torch.optim as optim

In [16]:
cat_dict = movielen.cat_dict
pos_names = movielen.pos_cat_names
neg_names = movielen.neg_cat_names
num_dim = 32

In [19]:
from models.torch_fm import TorchFM, FMLearner

In [20]:
model = TorchFM(cat_dict, pos_names, neg_names, num_dim=10)
model

TorchFM(
  (emb_linear_layer): ModuleList(
    (0): Embedding(943, 1)
    (1): Embedding(1413, 1)
    (2): Embedding(1415, 1)
  )
  (emb_factor_layer): ModuleList(
    (0): Embedding(943, 10)
    (1): Embedding(1413, 10)
    (2): Embedding(1415, 10)
  )
)

In [21]:
pos_preds, neg_preds = model(pos_batch, neg_batch)

In [22]:
print(pos_preds)
print(neg_preds)

tensor([ -2.3542, -15.5298,  23.6379,  -2.5658, -19.6251, -12.7302,   2.4559,
         -1.4607,   5.8175, -11.1045,  -2.1898,  -6.4191,  11.1794,   2.9016,
         -1.5356,  -4.3187], grad_fn=<SqueezeBackward0>)
tensor([-20.2431, -13.3625,  -7.2352,  -8.0666, -10.4701,  -5.1879,  -0.4698,
         -8.5503,  -0.9939,  -2.7569,  -8.5696,  -2.4610, -10.3936,  12.5140,
        -19.3883,  -7.7941], grad_fn=<SqueezeBackward0>)


In [23]:
learner = FMLearner(model, databunch)

In [24]:
bprloss = learner.criterion(pos_preds, neg_preds, linear_reg=0.001)
bprloss

tensor([44.6341], grad_fn=<NegBackward>)

In [28]:
binary_ranks = (pos_preds - neg_preds) > 0
binary_ranks = binary_ranks.to(T.float)
users_index = pos_batch[:, 0]
users, user_counts = T.unique(users_index, return_counts=True)
user_counts = user_counts.to(T.float)

In [32]:
print("user index: ", users_index)
print("user counts: ", user_counts)

user index:  tensor([850, 850, 850, 850, 850, 850, 850, 850, 850, 850, 711, 711, 711, 711,
        711, 711])
user counts:  tensor([ 6., 10.])


In [33]:
learner.user_counts.size()

torch.Size([943])

In [38]:
learner.user_counts[users] += user_counts

In [39]:
learner.user_counts[users]

tensor([12., 20.])

In [39]:
learner.metric(pos_batch, pos_preds, neg_preds)

tensor(0)

In [52]:
binary_ranks = (pos_preds - neg_preds) > 0
binary_ranks = binary_ranks.to(T.float)
binary_ranks

tensor([0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.])

In [53]:
users_index = pos_batch[:, 0]

In [54]:
users, user_counts = T.unique(users_index, return_counts=True)

In [55]:
print("users: {}".format(users))
print("user counts: {}".format(user_counts))

users: tensor([ 32,  48,  56, 233, 307, 392, 425, 436, 510, 534, 628, 675, 701, 822,
        891, 932])
user counts: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [56]:
users_index

tensor([675, 932,  48, 307, 628, 436, 701, 510, 233, 891, 392, 425,  56, 534,
         32, 822])

In [57]:
from torch_scatter import scatter_mean

In [58]:
auc_per_user = scatter_mean(binary_ranks, users_index)
auc_per_user

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [59]:
T.sum(auc_per_user) / users.size(0)

tensor(0.6875)